### Initialize Embedding model

In [1]:
from langchain_community.embeddings import OpenVINOEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "CPU"}
encode_kwargs = {"mean_pooling": True, "normalize_embeddings": True}

ov_embeddings = OpenVINOEmbeddings(
    model_name_or_path=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    show_progress=True,
)

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
/home/will/Downloads/coding/uni/offline_linux_helper/venv/lib/python3.13/site-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask


### Creating a vector store with ChromaDB (in memory, local)

In [2]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="arch_wiki_collection",
    embedding_function=ov_embeddings,
    persist_directory="chroma_langchain_db"
)

### Document loading and splitting

In [3]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader

directory_path = "./arch-wiki/html/en/"

if not os.path.exists(directory_path):
    print(f"Directory not found: {directory_path}")
else:
    loader = DirectoryLoader(
        path=directory_path,
        glob="**/*.html", # all html files and subdirectories
        loader_cls=TextLoader, # keeps html tags for document splitting
        show_progress=True
    )

docs = loader.load() # loads all documents at once

100%|█████████████████████████████████████████████████████████████████████▉| 2491/2493 [00:00<00:00, 11724.54it/s]


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

# initilaize header splitter
html_header_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# initilaize text splitter to ensure chunks fit in local llm
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

document_chunks=[]

for doc in docs:
    # if html file, split by header, then split again into smaller chunks
    if doc.metadata.get("source", "").endswith(".html"):
        header_splits = html_header_splitter.split_text(doc.page_content)

        # apply original source metadata to each split
        for split in header_splits:
            split.metadata["source"] = doc.metadata.get("source")

        # DEBUG
        # print(f"Found {len(header_splits)} header sections.")
        # if len(header_splits) > 0:
        #     print(f"Metadata of first section: {header_splits[0].metadata}")

        
        document_chunks.extend(text_splitter.split_documents(header_splits))
    else:
        # non-html, use recursive splitter directly
        document_chunks.extend(text_splitter.split_documents([doc]))

Found 63 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/WireGuard.html'}
Found 15 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Sugar.html'}
Found 26 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Character_encoding.html'}
Found 13 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Category:Hardware_detection_and_troubleshooting.html'}
Found 27 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Intel_Quartus_Prime.html'}
Found 46 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Dhcpcd.html'}
Found 27 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Guitarix.html'}
Found 25 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Bluetooth_mouse.html'}
Found 25 header sections.
Metadata of first section: {'source': 'arch-wiki/html/en/Command-line_shell.html'}
Found 17 header sections.
Metadata o

In [5]:
document_chunks[0].metadata

{'source': 'arch-wiki/html/en/WireGuard.html'}

In [6]:
len(document_chunks)

75185

### Adding Documents into Vectorstore

In [10]:
from uuid import uuid4

BATCH_SIZE = 500 # max batch size is 5461

for i in range(0, len(document_chunks), BATCH_SIZE):
    # slice chunk list into batches
    batch = document_chunks[i : i + BATCH_SIZE]
    
    # generate random universally unique identifier (UUID)
    batch_uuids = [str(uuid4()) for _ in range(len(batch))] 

    vector_store.add_documents(
        documents=batch,
        ids=batch_uuids
    )

    # DEBUG
    print(f"Added batch {i//BATCH_SIZE + 1} (Documents {i} to {i+len(batch)})")

Batches: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:10<00:00, 12.04it/s]


Added batch 1 (Documents 0 to 500)


Batches: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:09<00:00, 13.03it/s]


Added batch 2 (Documents 500 to 1000)


Batches: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:08<00:00, 14.96it/s]


...

Added batch 149 (Documents 74000 to 74500)


Batches: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:17<00:00,  7.31it/s]


Added batch 150 (Documents 74500 to 75000)


Batches: 100%|████████████████████████████████████████████████████████████████████| 47/47 [00:03<00:00, 13.44it/s]


Added batch 151 (Documents 75000 to 75185)


In [16]:
collection_count = vector_store._collection.count()
print(f"Total items in Chroma: {collection_count}")

Total items in Chroma: 7973


### Querying a document

In [12]:
results = vector_store.similarity_search_with_score( 
    # langchain embeds text query automatically
    "How to install minecraft?",
    k=2,
)

Batches: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 92.26it/s]


In [13]:
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.338752] Install [{'Header 3': 'Wake-on-LAN', 'Header 2': 'Install', 'source': 'arch-wiki/html/en/OpenVPN/Checklist_guide.html', 'Header 1': 'dm-crypt/Encrypting an entire system'}]
* [SIM=0.360615] 


Minecraft - ArchWiki













Jump to content












Contents
move to sidebar
hide




Beginning





1
Client




Toggle Client subsection





1.1
Java Edition






1.1.1
Installation








1.1.2
Firewall configuration for Client/LAN worlds










1.2
Bedrock Edition








1.3
Minecraft Education










2
Server




Toggle Server subsection





2.1
Java Edition








2.2
Bedrock Edition






2.2.1
Installation








2.2.2
Configuration












3
Minecraft mod launchers








4
Other programs and editors








5
Troubleshooting




Toggle Troubleshooting subsection





5.1
Logs








5.2
Client or server does not start








5.3
Broken fonts with MinecraftForge








5.4
MultiMC forks unable to build








5.5
Cannot change pulseaudio s